This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [19]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

AyBRycntfmq8UyMnKTnc


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [20]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [21]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
ticker = "/AFX_X"
dataset = "/FSE"
start = "2014-08-12"
end = "2014-08-12"
data_format = "/data." + "json"
url = r"https://data.nasdaq.com/api/v3/datasets" + dataset + ticker + data_format
payload = {"api_key":API_KEY, "start_date":start, "end_date":end}
r = requests.get(url, params=payload)

In [22]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r_data = r.json()
# r.json().keys()
r_data

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2014-08-12',
  'end_date': '2014-08-12',
  'frequency': 'daily',
  'data': [['2014-08-12',
    21.97,
    22.11,
    21.93,
    22.08,
    None,
    16181.0,
    356343.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

#### Q1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [23]:
# Pull data for 2017
start = "2017-01-01"
end = "2017-12-31"
payload = {"api_key":API_KEY, "start_date":start, "end_date":end}
r = requests.get(url, params=payload)

#### Q2. Convert the returned JSON object into a Python dictionary.

In [ ]:
# Convert to dict
r_json = r.json()
r_meta = r_json['dataset_data']
r_data = r_meta['data']

# Map column names to list indices
column_map = {name:i for i, name in enumerate(r_meta['column_names'])}

#### Q3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [24]:
# Find high and low prices
open_prices = [x[column_map['Open']] for x in r_data]

# Remove extra None values
open_prices = [x for x in open_prices if x is not None]
print(f'Highest opening: {max(open_prices)}')
print(f'Lowest opening: {min(open_prices)}')

Highest opening: 53.11
Lowest opening: 34.0


#### Q4. What was the largest change in any one day (based on High and Low price)?

In [25]:
# Find largest change
highs = [x[column_map['High']] for x in r_data]
lows = [x[column_map['Low']] for x in r_data]
delta = [x - y for x,y in zip(highs, lows)]
print(f'Largest Change in a Single Day: {round(max(delta), 4)}')  

Largest Change in a Single Day: 2.81


#### Q5. What was the largest change between any two days (based on Closing Price)?

In [26]:
# Find largest change between two closing prices
closings = [x[column_map['Close']] for x in r_data]
changes = [abs(closings[i] - closings[i+1]) for i in range(len(closings)-2)]
print(f'Largest Change between Two Days: {round(max(changes),4)}')

Largest Change between Two Days: 2.56


#### Q6. What was the average daily trading volume during this year?

In [27]:
# Find largest daily trading volume
import statistics
volumes = [x[column_map['Traded Volume']] for x in r_data]
print(f'Average Daily Trading Volume: {statistics.mean(volumes)}')

Average Daily Trading Volume: 89124.33725490196


#### Q7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [28]:
def median(data):
    sorted_data = sorted(data)
    n_values = len(data)
    if n_values % 2 == 0:
        lower_middle = n_values // 2 - 1
        upper_middle = n_values // 2
        return (sorted_data[lower_middle] + sorted_data[upper_middle]) / 2
    else:
        return sorted_data[int((n_values - 1) / 2)]

print(f'Median Trading Volume: {median(volumes)}')

Median Trading Volume: 76286.0
